# 02. Building a PySC2 Agent (with jupyter notebook)

* 싸이그래머 / DeepStar - PySC2
* 김무성

# 차례
* Remind : How to use like Gym
* Building a Basic PySC2 Agent

---------------------------

# Remind : How to use like Gym

* [3] How to train StarCraft II Bots - https://colab.research.google.com/drive/1AzCKV98UaQQz2aJIeGWlExcxBrpgKsIV
* [4] chagmgang's Reinforcement Learning with StarCraft II - https://drive.google.com/file/d/10641_2vgu937r0FjxxArC1fZTRv5aH1n/view
* [5] Build a Zerg Bot with PySC2 2.0 - https://itnext.io/build-a-zerg-bot-with-pysc2-2-0-295375d2f58e
(github) 
    - How To: PySC2 - https://github.com/skjb/pysc2-tutorial

## Setting Map

### Setting Counfigure

##### 참고자료
* [3] How to train StarCraft II Bots - https://colab.research.google.com/drive/1AzCKV98UaQQz2aJIeGWlExcxBrpgKsIV


In [1]:
# unfortunately, PySC2 uses Abseil, which treats python code as if its run like an app
# This does not play well with jupyter notebook
# So we will need to monkeypatch sys.argv


import sys
sys.argv = ["python", "--map", "Simple64"]

In [2]:
from absl import app
from absl import flags

In [3]:
from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.lib import point_flag
from pysc2.lib import stopwatch

In [4]:
FLAGS = flags.FLAGS

In [5]:
# because of Abseil's horrible design for running code underneath Colabs
# We have to pull out this ugly hack from the hat
if "flags_defined" not in globals():
    flags.DEFINE_bool("render", True, "Whether to render with pygame.")
    point_flag.DEFINE_point("feature_screen_size", "84",
                            "Resolution for screen feature layers.")
    point_flag.DEFINE_point("feature_minimap_size", "64",
                            "Resolution for minimap feature layers.")
    point_flag.DEFINE_point("rgb_screen_size", None,
                            "Resolution for rendered screen.")
    point_flag.DEFINE_point("rgb_minimap_size", None,
                            "Resolution for rendered minimap.")
    flags.DEFINE_enum("action_space", None, sc2_env.ActionSpace._member_names_,  # pylint: disable=protected-access
                      "Which action space to use. Needed if you take both feature "
                      "and rgb observations.")
    flags.DEFINE_bool("use_feature_units", False,
                      "Whether to include feature units.")
    flags.DEFINE_bool("disable_fog", False, "Whether to disable Fog of War.")

    flags.DEFINE_integer("max_agent_steps", 0, "Total agent steps.")
    flags.DEFINE_integer("game_steps_per_episode", None, "Game steps per episode.")
    flags.DEFINE_integer("max_episodes", 0, "Total episodes.")
    flags.DEFINE_integer("step_mul", 8, "Game steps per agent step.")

    flags.DEFINE_string("agent", "pysc2.agents.random_agent.RandomAgent",
                        "Which agent to run, as a python path to an Agent class.")
    flags.DEFINE_enum("agent_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 1's race.")

    flags.DEFINE_string("agent2", "Bot", "Second agent, either Bot or agent class.")
    flags.DEFINE_enum("agent2_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 2's race.")
    flags.DEFINE_enum("difficulty", "very_easy", sc2_env.Difficulty._member_names_,  # pylint: disable=protected-access
                      "If agent2 is a built-in Bot, it's strength.")

    flags.DEFINE_bool("profile", False, "Whether to turn on code profiling.")
    flags.DEFINE_bool("trace", False, "Whether to trace the code execution.")
    flags.DEFINE_integer("parallel", 1, "How many instances to run in parallel.")

    flags.DEFINE_bool("save_replay", True, "Whether to save a replay at the end.")

    flags.DEFINE_string("map", None, "Name of a map to use.")
    flags.mark_flag_as_required("map")

In [6]:
flags_defined = True

In [7]:
FLAGS(sys.argv)

['python']

In [8]:
players = []
players.append(sc2_env.Agent(sc2_env.Race[FLAGS.agent_race]))
print(players)

[Agent(race=<Race.random: 4>, name='<unknown>')]


In [9]:
map_name = FLAGS.map
visualize = True
print(map_name)
print(visualize)

Simple64
True


------------------------------

## Random Agent Example

##### 참고자료
* [5] Build a Zerg Bot with PySC2 2.0 - https://itnext.io/build-a-zerg-bot-with-pysc2-2-0-295375d2f58e
(github) 
    - How To: PySC2 - https://github.com/skjb/pysc2-tutorial

In [10]:
from pysc2.agents import random_agent

In [11]:
import pysc2.agents

In [12]:
dir(pysc2.agents)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'base_agent',
 'random_agent']

In [13]:
with sc2_env.SC2Env(
          map_name=map_name,
          players=players,
          agent_interface_format=sc2_env.parse_agent_interface_format(
              feature_screen=FLAGS.feature_screen_size,
              feature_minimap=FLAGS.feature_minimap_size,
              rgb_screen=FLAGS.rgb_screen_size,
              rgb_minimap=FLAGS.rgb_minimap_size,
              action_space=FLAGS.action_space,
              use_feature_units=FLAGS.use_feature_units),
          step_mul=FLAGS.step_mul,
          game_steps_per_episode=FLAGS.game_steps_per_episode,
          disable_fog=FLAGS.disable_fog,
          visualize=visualize) as env :

    agent = random_agent.RandomAgent()
    agent.setup(env.observation_spec()[0], env.action_spec()[0])
    
    done = False
    global_step = 0
    
    obs = env.reset()
    
    while not done : 
        global_step += 1

        #print(obs)
        action = agent.step(obs[0])
        print(action)
        obs = env.step(actions=[action])
       
        if global_step == 10:
            done = True

/Users/moodern/anaconda3/envs/deepstar/lib/python3.5/site-packages/pysc2/lib/features.py:862: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for f in SCREEN_FEATURES),
/Users/moodern/anaconda3/envs/deepstar/lib/python3.5/site-packages/pysc2/lib/features.py:867: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for f in MINIMAP_FEATURES),


FunctionCall(function=2, arguments=[[2], [3, 6]])
FunctionCall(function=3, arguments=[[0], [66, 10], [43, 42]])
FunctionCall(function=1, arguments=[[14, 41]])
FunctionCall(function=1, arguments=[[55, 59]])
FunctionCall(function=0, arguments=[])
FunctionCall(function=1, arguments=[[20, 17]])
FunctionCall(function=2, arguments=[[2], [43, 18]])
FunctionCall(function=0, arguments=[])
FunctionCall(function=3, arguments=[[1], [65, 4], [67, 75]])
FunctionCall(function=0, arguments=[])


W0401 19:17:25.793968 140736103465792 sc_process.py:204] Killing the process.


-----------------------

# Building a Basic PySC2 Agent

## 1. Creating an Agent

SimpleAgent 클래스를 아래처럼 만들자

In [14]:
from pysc2.agents import base_agent
from pysc2.lib import actions

class SimpleAgent(base_agent.BaseAgent):
    def step(self, obs):
        super(SimpleAgent, self).step(obs)
        
        return actions.FunctionCall(actions.FUNCTIONS.no_op.id, [])

In [15]:
with sc2_env.SC2Env(
          map_name=map_name,
          players=players,
          agent_interface_format=sc2_env.parse_agent_interface_format(
              feature_screen=FLAGS.feature_screen_size,
              feature_minimap=FLAGS.feature_minimap_size,
              rgb_screen=FLAGS.rgb_screen_size,
              rgb_minimap=FLAGS.rgb_minimap_size,
              action_space=FLAGS.action_space,
              use_feature_units=FLAGS.use_feature_units),
          step_mul=FLAGS.step_mul,
          game_steps_per_episode=FLAGS.game_steps_per_episode,
          disable_fog=FLAGS.disable_fog,
          visualize=visualize) as env :

    agent = SimpleAgent()
    agent.setup(env.observation_spec()[0], env.action_spec()[0])
    
    done = False
    global_step = 0
    
    obs = env.reset()
    
    while not done : 
        global_step += 1

        #print(obs)
        action = agent.step(obs[0])
        print(action)
        obs = env.step(actions=[action])
       
        if global_step == 10:
            done = True

/Users/moodern/anaconda3/envs/deepstar/lib/python3.5/site-packages/pysc2/lib/features.py:862: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for f in SCREEN_FEATURES),
/Users/moodern/anaconda3/envs/deepstar/lib/python3.5/site-packages/pysc2/lib/features.py:867: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for f in MINIMAP_FEATURES),


FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])
FunctionCall(function=<_Functions.no_op: 0>, arguments=[])


### 참고) simple_agent.py 파일을 만들어서 실행할 수도 있다.

In [16]:
%%writefile simple_agent.py
from pysc2.agents import base_agent
from pysc2.lib import actions

class SimpleAgent(base_agent.BaseAgent):
    def step(self, obs):
        super(SimpleAgent, self).step(obs)
        
        return actions.FunctionCall(actions.FUNCTIONS.no_op.id, [])

Overwriting simple_agent.py


In [17]:
%ls

02_pysc2_basic_01_env_states_actions_with_jupyter.ipynb
02_pysc2_basic_02_building_pysc2_agent_with_jupyter.ipynb
simple_agent.py


In [20]:
!python -m pysc2.bin.agent --map Simple64 --agent simple_agent.SimpleAgent --agent_race terran

I0401 19:22:01.851667 140736103465792 sc_process.py:110] Launching SC2: /Applications/StarCraft II/Versions/Base72282/SC2.app/Contents/MacOS/SC2 -listen 127.0.0.1 -port 18367 -dataDir /Applications/StarCraft II/ -tempDir /var/folders/5n/qdpyvzc169jbh39gdpl_yfsc0000gn/T/sc-9jbeo43x/ -displayMode 0 -windowwidth 640 -windowheight 480 -windowx 50 -windowy 50
I0401 19:22:01.859534 140736103465792 remote_controller.py:163] Connecting to: ws://127.0.0.1:18367/sc2api, attempt: 0, running: True
err = 3 /opt/blizzard/buildserver/data.noindex/repositories/sc2/branches/SC2.4.8.b/Game/Contrib/macteam/libs/ClampCursor/Contrib/mach_override/mach_override.c:244
err = 3 /opt/blizzard/buildserver/data.noindex/repositories/sc2/branches/SC2.4.8.b/Game/Contrib/macteam/libs/ClampCursor/Contrib/mach_override/mach_override.c:258
err = 3 /opt/blizzard/buildserver/data.noindex/repositories/sc2/branches/SC2.4.8.b/Game/Contrib/macteam/libs/ClampCursor/Contrib/mach_override/mach_override.c:263
NSWindowEdgeResizing

I0401 19:22:42.509917 140736103465792 sc2_env.py:656] Environment Close
I0401 19:22:42.510200 140736103465792 sc2_env.py:656] Environment Close


## 2. Controlling an SCV

먼저 코드 작성이 편하도록 몇 가지 상수를 정의.

In [22]:
from pysc2.agents import base_agent
from pysc2.lib import actions
from pysc2.lib import features

In [23]:
# Functions
_BUILD_SUPPLYDEPOT = actions.FUNCTIONS.Build_SupplyDepot_screen.id
_NOOP = actions.FUNCTIONS.no_op.id
_SELECT_POINT = actions.FUNCTIONS.select_point.id

# Features
_PLAYER_RELATIVE = features.SCREEN_FEATURES.player_relative.index
_UNIT_TYPE = features.SCREEN_FEATURES.unit_type.index

# Unit IDs
_TERRAN_COMMANDCENTER = 18
_TERRAN_SCV = 45

# Parameters
_PLAYER_SELF = 1
_NOT_QUEUED = [0]
_QUEUED = [1]

On the Simple64 map there are two spawn locations, one at the top left, and another at the bottom right. In order to build and rally units relative to these positions we first need to detect where we have spawned. Just as a human would, we look at the mini map to determine where our units are located. Let’s add a variable to store our location:
(구글 번역 :
Simple64 맵에는 두 개의 스폰 위치가 있습니다. 하나는 왼쪽 상단에, 다른 하나는 오른쪽 하단에 있습니다. 이러한 위치와 관련하여 유닛을 구축하고 집계하기 위해서는 먼저 우리가 어디에서 스폰했는지 탐지해야합니다. 인간처럼 우리는 미니 맵을보고 유닛이 어디에 위치해 있는지 확인합니다. 우리 위치를 저장할 변수를 추가합시다.)

```python
class SimpleAgent(base_agent.BaseAgent):
   base_top_left = None
```

The mini map is 64 units across and 64 units tall, with (0, 0) being the top left coordinate. Values we receive are 0 indexed, so we can assume that anything with an x or y coordinate between 0 and 31 is at the top left. Anything with an x or y coordinate between 32 and 63 is at the bottom right.
(구글 번역 :
미니지도는 가로 세로 64 단위이고 높이는 64 단위이며, 왼쪽 상단 좌표는 (0, 0)입니다. 우리가받은 값은 0으로 인덱싱되므로, 0과 31 사이 의 x 또는 y 좌표 를 갖는 것이 왼쪽 상단에 있다고 가정 할 수 있습니다 . 32와 63 사이 의 x 또는 y 좌표를 가진 것은 오른쪽 하단에 있습니다.)

One of the observation sets is “player relative” data. This array contains a list of units arranged “relative” to the current player. In this case we want to find all units that belong to the current player, then we can get the mean of their coordinates to roughly determine where our base is. Since the first parameter is the y coordinate, we can use that:
(구글 번역 : 
관찰 세트 중 하나는 "플레이어 상대"데이터입니다. 이 배열은 현재 플레이어에게 "상대적으로"배열 된 유닛 목록을 포함합니다. 이 경우 현재 플레이어에 속한 모든 유닛을 찾고 싶습니다. 좌표 의 평균 을 구하면 근본적인 위치를 대략적으로 결정할 수 있습니다. 첫 번째 매개 변수는 y 좌표이므로 다음과 같이 사용할 수 있습니다.)


```python
def step(self, obs):
        super(SimpleAgent, self).step(obs)
        
        time.sleep(0.5)
        
        if self.base_top_left is None:
            player_y, player_x = (obs.observation["minimap"][_PLAYER_RELATIVE] == _PLAYER_SELF).nonzero()
```

In [ ]:
class SimpleAgent(base_agent.BaseAgent):
    base_top_left = None
    
    def step(self, obs):
        super(SimpleAgent, self).step(obs)
        
        time.sleep(0.5)

        if self.base_top_left is None:
            player_y, player_x = (obs.observation["minimap"][_PLAYER_RELATIVE] == _PLAYER_SELF).nonzero()
        
        return actions.FunctionCall(actions.FUNCTIONS.no_op.id, [])

# 참고자료
* [1] pysc2 environments doc - https://github.com/deepmind/pysc2/blob/master/docs/environment.md
* [2] Train your first Starcraft II learning agent - 
https://soygema.github.io/starcraftII_machine_learning
* [3] How to train StarCraft II Bots - https://colab.research.google.com/drive/1AzCKV98UaQQz2aJIeGWlExcxBrpgKsIV
* [4] chagmgang's Reinforcement Learning with StarCraft II - https://drive.google.com/file/d/10641_2vgu937r0FjxxArC1fZTRv5aH1n/view
* [5] Build a Zerg Bot with PySC2 2.0 - https://itnext.io/build-a-zerg-bot-with-pysc2-2-0-295375d2f58e
     - (github) How To: PySC2 - https://github.com/skjb/pysc2-tutorial